In [1]:
import ccxt, abc, time, ssl
from statistics import mean
from threading import Thread, Event
from matplotlib import pyplot as pl

updated = Event()

In [2]:
class Exchange(ccxt.coinsecure):
    def __init__(self):
        self.data = None
        super().__init__()
        
    def check_for_update(self):
        def thread_exec():
            prevTimestamp = None
            while True:
                try:
                    response = self.fetchTicker('BTC/INR')
                    if response['timestamp'] != prevTimestamp:
                        self.data, prevTimestamp = response['info'], response['timestamp']
                        print(self.data)
                        updated.set()
                except ssl.SSLEOFError: print('Non-fatal error occured.')
                time.sleep(2)
        thread = Thread(target=thread_exec)
        thread.start()

class Classifier(abc.ABC):
    def __init__(self, exchange): self.exchange = exchange
    
    @abc.abstractmethod
    def predict(self): pass

In [3]:
class _Window(list):
    def __init__(self, window_size=20):
        self.__window_size, self.__actual_window_size = window_size, 0
        super().__init__()
        
    def append(self, price):
        super().append(price)
        if self.__window_size > self.__actual_window_size: self.__actual_window_size += 1
        else: del self[0]
            
class SMA(Classifier):
    def __init__(self, exchange, *, window_size=20):
        self.__window_size, self.predict_sp, self.predict_bp = window_size, None, None
        self.__sp_window, self.__bp_window = _Window(window_size), _Window(window_size)
        super().__init__(exchange)
    
    def predict(self):
        def classify(avg, price):
            if price < avg: return -1
            if price > avg: return 1
            return 0
        
        def thread_exec():
            sp, bp = None, None
            while True:
                updated.wait()
                if sp != self.exchange.data['ask']:
                    sp = self.exchange.data['ask']
                    self.__sp_window.append(sp)
                    predict_sp = classify(mean(self.__sp_window), sp)
                if bp != self.exchange.data['bid']:
                    bp = self.exchange.data['bid']
                    self.__bp_window.append(bp)
                    predcit_bp = classify(mean(self.__bp_window), bp)
                print(self.__sp_window, self.__bp_window)
                print('sp: {}'.format(predict_sp), 'bp: {}'.format(predcit_bp))
                updated.clear()
        thread = Thread(target=thread_exec)
        thread.start()

In [4]:
exchange = Exchange()

In [5]:
exchange.checkForUpdate()

In [6]:
predictor = SMA(exchange, window_size=3)

In [7]:
predictor.predict()

In [8]:
class Plot:
    def __init__(self, data, predicted):
        self.fig, self.data, self.predicted = pl.figure('Live'), data, predicted
        self.tick = [data]

    def update(self):
        def thread_exec():
            while True:
                updated.wait()
                self.tick.append(data)
                pl.plot(tick)
                pl.scatter(predicted)
                self.fig.canvas.draw()
                updated.clear()
        thread = Thread(target=thread_exec)
        thread.start()

{'lastPrice': 58500000, 'timestamp': 1518518778320, 'bid': 57200000, 'ask': 58499800, 'fiatvolume': 342324301, 'coinvolume': 589400000, 'open': 58299897, 'high': 59000000, 'low': 57175099}
[58499800] [57200000]
sp: 0 bp: 0


In [9]:
Plot(exchange.data['bid'], predictor.predict_bp).update()
pl.show()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\program files\python36\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\program files\python36\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "c:\program files\python36\lib\http\client.py", line 1331, in getresponse
    response.begin()
  File "c:\program files\python36\lib\http\client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "c:\program files\python36\lib\http\client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "c:\program files\python36\lib\socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "c:\program files\python36\lib\ssl.py", line 1009, in recv_into
    return self.read(nbytes, buffer)
  File "c:\program files\py